In [1]:
import numpy as np
import cv2
from keras.models import load_model

2023-07-29 14:32:49.643640: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
# Load face cascade
face_cascade = cv2.CascadeClassifier('../xml_files/haarcascade_frontalface_default.xml')

# Load the model
model = load_model('../final_model/keras_model.h5')

# CAMERA can be 0 or 1 based on default camera of your computer.
# Next line may be used in order to get input from the camera.
camera = cv2.VideoCapture(0)
# camera.release()

# Grab the labels from the labels.txt file. This will be used later.
labels = open('../final_model/labels.txt', 'r').readlines()

## Image Input

In [17]:
# CROP THE FACE:
image_name = "data13.jpg"
input_path = f'../data/{image_name}'
output_path = f'../test_outputs/'

# Load the image and convert it to grayscale
image = cv2.imread(input_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
if (len(faces) == 0):
    print("Face Not Found!")

cropped_faces = []
# Iterate over the detected faces
face_index = 0
for (x,y,w,h) in faces:
    # Crop the image (0.5 * length_face) cm above the face
    y_offset = int(0.5 * h)  # 0.5 * length_face above the face
    crop_img = image[y-y_offset:y+h, x:x+w]

    # Save the cropped image to a file
    try:
        cv2.imwrite(f"{output_path}cropped_{face_index}_{image_name}", crop_img)
        cropped_faces.append(crop_img)
    except:
        pass
    
    face_index +=1

In [14]:
# DETECT WHETHER THE FACE HAS HELMET:

for image in cropped_faces:
    # Resize the raw image into (224-height,224-width) pixls.
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # DEBUG
    cv2.imwrite(f"../test_preprocess/image/resized_{image_name}", image)
    # DEBUG
      
    # Make the image a numpy array and reshape it to the models input shape.
    image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)
    
    # Normalize the image array
    image = (image / 127.5) - 1
    
    # Have the model predict what the current image is. Model.predict
    # returns an array of percentages. Example:[0.2,0.8] meaning its 20% sure
    # it is the first label and 80% sure its the second label.
    probabilities = model.predict(image)
    # Print what the highest value probabilitie label
    print(f"Sureness level: {max(probabilities[0]) * 100} %")
    print(labels[np.argmax(probabilities)])
    

1/1 [==============================] - 0s 31ms/step
Sureness level: 65.43623208999634 %
1 No Helmet



## Camera Input

In [18]:
# CROP THE FACE:

# Number of iterations:
num_iterations = 100

# cam
camera = cv2.VideoCapture(0)

while True:
    # Load the image and convert it to grayscale
    ret, image = camera.read()
    
    if ret == False:
        continue
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow('webcam', image)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if (len(faces) == 0):
        print("Face Not Found!")

    cropped_faces = []
    # Iterate over the detected faces
    face_index = 0
    for (x,y,w,h) in faces:
        # Crop the image (0.5 * length_face) cm above the face
        y_offset = int(0.5 * h)  # 0.5 * length_face above the face
        crop_img = image[y-y_offset:y+h, x:x+w]

        # Save the cropped image to a file
        try:
            cropped_faces.append(crop_img)
        except:
            pass
        
        face_index +=1

    # DETECT WHETHER THE FACE HAS HELMET:

    for image in cropped_faces:
        # Resize the raw image into (224-height,224-width) pixels.
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

        # DEBUG
        cv2.imwrite("../test_preprocess/camera/resized_camera.jpg", image)
        # DEBUG
      

        # Make the image a numpy array and reshape it to the models input shape.
        image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)
        # Normalize the image array
        image = (image / 127.5) - 1
        # Have the model predict what the current image is. Model.predict
        # returns an array of percentages. Example:[0.2,0.8] meaning its 20% sure
        # it is the first label and 80% sure its the second label.
        probabilities = model.predict(image)
        # Print what the highest value probabilitie label
        print(f"Sureness level: {max(probabilities[0]) * 100} %")
        print(labels[np.argmax(probabilities)])
    
    if (cv2.waitKey(1) == 27):
       break
    
camera.release()
cv2.destroyAllWindows()

Face Not Found!
1/1 [==============================] - 1s 1s/step
Sureness level: 93.78589391708374 %
1 No Helmet

1/1 [==============================] - 0s 35ms/step
Sureness level: 95.14492154121399 %
1 No Helmet

1/1 [==============================] - 0s 31ms/step
Sureness level: 95.23208737373352 %
1 No Helmet

1/1 [==============================] - 0s 34ms/step
Sureness level: 96.35847210884094 %
1 No Helmet

1/1 [==============================] - 0s 34ms/step
Sureness level: 97.99647331237793 %
1 No Helmet

1/1 [==============================] - 0s 33ms/step
Sureness level: 97.3625898361206 %
1 No Helmet

1/1 [==============================] - 0s 33ms/step
Sureness level: 95.84303498268127 %
1 No Helmet

1/1 [==============================] - 0s 38ms/step
Sureness level: 92.7346408367157 %
1 No Helmet

1/1 [==============================] - 0s 35ms/step
Sureness level: 93.68639588356018 %
1 No Helmet

1/1 [==============================] - 0s 36ms/step
Sureness level: 95.44497132

KeyboardInterrupt: 

: 